Databricks Auto Loader is a feature that automatically and incrementally ingests new files from cloud storage (like ADLS Gen2, S3, or GCS) into Delta tables. You use it because it’s scalable, efficient, and reliable for handling continuous data ingestion without reprocessing old files


### **Dynamic Capabilities**

In [0]:
dbutils.widgets.text("file_name","")

In [0]:
p_file_name = dbutils.widgets.get("file_name")

### **Data Reading**

In [0]:
df=spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format","parquet")\
        .option("cloudFiles.schemalocation",f"abfss://bronze@databricksadlsete.dfs.core.windows.net/checkpoint_{p_file_name}")\
        .load(f"abfss://source@databricksadlsete.dfs.core.windows.net/{p_file_name}")


### **Data Writting**

In [0]:
df.writeStream.format("parquet")\
    .outputMode("append")\
    .option("checkpointLocation",f"abfss://bronze@databricksadlsete.dfs.core.windows.net/checkpoint_{p_file_name}")\
    .option("path",f"abfss://bronze@databricksadlsete.dfs.core.windows.net/{p_file_name}")\
    .trigger(once=True)\
    .start()

In [0]:
df=spark.read.format("parquet").load(f"abfss://bronze@databricksadlsete.dfs.core.windows.net/{p_file_name}")
df.count()